Insper

# Aula 08 - Spark ML - Machine Learning com Spark

Vamos fazer o setup de nosso ambiente Spark.

In [3]:
# Criar a sessao do Spark
from pyspark.sql import SparkSession
spark = SparkSession \
            .builder \
            .master("local[*]") \
            .appName("MUDE_AQUI") \
            .getOrCreate()

## Modelo de regressão linear

Vamos usar o clássico conjunto de dados de Advertising para predizer o número de vendas de um produto, em função dos valores gastos em campanhas de TV, Radio e Jornal.

Iniciamos com a leitura do conjunto de dados.

In [7]:
data = spark.read.csv('Advertising.csv',
                      header=True,
                      inferSchema=True)

Exiba as primeiras cinco linhas do dataset.

In [9]:
data = data.drop("_c0")

Remova a coluna `_c0`.

In [10]:
data.describe().show()

+-------+-----------------+------------------+------------------+------------------+
|summary|               TV|             Radio|         Newspaper|             Sales|
+-------+-----------------+------------------+------------------+------------------+
|  count|              200|               200|               200|               200|
|   mean|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|              0.7|               0.0|               0.3|               1.6|
|    max|            296.4|              49.6|             114.0|              27.0|
+-------+-----------------+------------------+------------------+------------------+



Exiba novamente as primeiras cinco linhas.

In [14]:
data.show(5)

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows



Vamos agora exibir as principais estatísticas descritivas do conjunto de dados.

O Spark faz uso de uma interface similar ao Scikit-Learn para desenvolver modelos preditivos. Baseado no conceito de `transformer`, nós vamos transformando o dataset em outro dataset com os ajustes necessários para o desenvolvimento de nosso modelo.

A estrutura de dados mais utilizada para o desenvolvimento de modelos é o `Vector` que possui, dentre outras funções, um bom suporte para dados esparsos.

Agora vamos importar o `VectorAssembler` que está em `pyspark.ml.feature` e também o modelo de regrssão linear (`LinearRegression`) que está em `pyspark.ml.regression`.

In [29]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

Vamos dividir os dados no conjunto de treino (70%) e teste (30%), como segue:

In [18]:
train, test = data.randomSplit([0.7,0.3], seed = 42)

Para desenvolver seu modelo em Spark, você precisará ter uma coluna denominada `features`, que será resultado de todos os processos de transformação de dados necessários para o correto desenvolvimento dos modelos.

Vamos criar uma variável chamada `vec` que será o nosso `VectorAssembler`. Devemos informar quais colunas serão concatenadas em um `vector` e qual será o nome desse `vector`.

In [27]:
data.columns[:-1] # tudo meno o ultimo valor

['TV', 'Radio', 'Newspaper']

Feito isso, podemos ver o resultado usando a função `transform()`.

In [30]:
vec = VectorAssembler(inputCols=data.columns[:-1],outputCol="Features")

In [31]:
train = vec.transform(train)

In [32]:
test = vec.transform(test)

Exiba as cinco primeiras linhas de train e, posteriormente, de teste.

In [33]:
train.show(5)

+---+-----+---------+-----+---------------+
| TV|Radio|Newspaper|Sales|       Features|
+---+-----+---------+-----+---------------+
|0.7| 39.6|      8.7|  1.6| [0.7,39.6,8.7]|
|4.1| 11.6|      5.7|  3.2| [4.1,11.6,5.7]|
|7.3| 28.1|     41.4|  5.5|[7.3,28.1,41.4]|
|7.8| 38.9|     50.6|  6.6|[7.8,38.9,50.6]|
|8.4| 27.2|      2.1|  5.7| [8.4,27.2,2.1]|
+---+-----+---------+-----+---------------+
only showing top 5 rows



In [34]:
test.show(5)

+----+-----+---------+-----+----------------+
|  TV|Radio|Newspaper|Sales|        Features|
+----+-----+---------+-----+----------------+
| 5.4| 29.9|      9.4|  5.3|  [5.4,29.9,9.4]|
| 8.6|  2.1|      1.0|  4.8|   [8.6,2.1,1.0]|
|11.7| 36.9|     45.2|  7.3|[11.7,36.9,45.2]|
|13.1|  0.4|     25.6|  5.3| [13.1,0.4,25.6]|
|17.2| 45.9|     69.3|  9.3|[17.2,45.9,69.3]|
+----+-----+---------+-----+----------------+
only showing top 5 rows



É possível observar a coluna `features`. Veja o schema do dataframe.

In [35]:
train.printSchema()

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Features: vector (nullable = true)



Agora chegou a vez de criarmos o nosso modelo de regressão linear. Você precisa informar dois parâmetros: `featureCol` (qual é a coluna que possui um vector das features) e `labelCol` que representa a coluna que possui a variável dependente.

In [38]:
lr = LinearRegression(featuresCol = "Features", labelCol ="Sales")

Análogo ao `scikit-learn`, use o método `fit()`

In [39]:
lrModel = lr.fit(train)

O atributo `coefficients` e o atributo `intercept` apresentam, respectivamente, os coeficientes da regressão linear e o valor do intercepto.

In [40]:
lrModel.coefficients

DenseVector([0.0459, 0.2009, -0.0035])

In [41]:
lrModel.intercept

2.735554921884289

E podemos usar a função `evaluate` para criar uma variável que nos permitirá obter as métricas comuns de desempenho do modelo.

In [47]:
evaluation_summary = lrModel.evaluate(test)

print("MAR : ", evaluation_summary.meanAbsoluteError)
print("RMSE : ", evaluation_summary.rootMeanSquaredError)
print("R2 : ", evaluation_summary.r2)


MAR :  1.3365541938281162
RMSE :  1.6614200921535163
R2 :  0.8651044648239229


# Profissionalizando nossos modelos com Pipeline


De fato, há inúmeros procedimentos que fazemos com os dados antes de treinar um modelo. Há processos de limpeza, padronização, one hot encoding, etc.

Veja nesse link [https://spark.apache.org/docs/latest/ml-features](https://spark.apache.org/docs/latest/ml-features) as mais diversa funções que podemos fazer nos dados com o Spark. Há códigos de exemplo para lhe auxiliar no aprendizado.

Vamos comecar importando `Pipeline` que está em `pyspark.ml`. E com isso vamos definir um pipeline formado por 2 estágios: vec (transforma as features em vector) e lr (nosso modelo de regressão linear)

In [49]:
from pyspark.ml import Pipeline

E agora criamos nosso pipeline

In [51]:
pipeline = Pipeline(stages=[vec,lr])

Vamos dividir novamente os dados em treino e teste

In [53]:
train, test = data.randomSplit([0.7,0.3],seed=42)

Agora podemos usar a função `fit()` do pipeline

In [54]:
pipelineModel = pipeline.fit(train)

Execute a função `transform` sob o conjunto de teste (`test`) e salve em `pred`.

In [55]:
pred = pipelineModel.transform(test)

In [56]:
pred.show(5)

+----+-----+---------+-----+----------------+------------------+
|  TV|Radio|Newspaper|Sales|        Features|        prediction|
+----+-----+---------+-----+----------------+------------------+
| 5.4| 29.9|      9.4|  5.3|  [5.4,29.9,9.4]| 8.958660646510964|
| 8.6|  2.1|      1.0|  4.8|   [8.6,2.1,1.0]| 3.548735670229508|
|11.7| 36.9|     45.2|  7.3|[11.7,36.9,45.2]| 10.52918016473767|
|13.1|  0.4|     25.6|  5.3| [13.1,0.4,25.6]|3.3276257934037154|
|17.2| 45.9|     69.3|  9.3|[17.2,45.9,69.3]|12.505787163829279|
+----+-----+---------+-----+----------------+------------------+
only showing top 5 rows



## Melhorando nosso pipeline com feature engineering

Lembre-se desse link para ver as mais usadas transformações nos dados: [https://spark.apache.org/docs/latest/ml-features](https://spark.apache.org/docs/latest/ml-features)


Vamos desenvolver outro modelo preditivo. Agora para predizer o gasto em planos de saúde. Teremos variáveis categóricas e contínuas.

Para as variáveis categóricas, vamos ver como aplicar `OneHotEncoding`.

In [58]:
gasto = spark.read.csv('gasto.csv', header=True, inferSchema=True)

Exiba as cinco primeiras linhas

In [60]:
gasto.show(5)

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
+---+------+------+--------+------+---------+-----------+
only showing top 5 rows



Exiba o schema.

In [61]:
gasto.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)



Separe agora em treino (70%) e teste (30%)

In [62]:
train, test = gasto.randomSplit([0.7,0.3], seed = 42)

Vamos agora definir as variáveis que são categóricas e quais são numéricas

In [64]:
numeric_cols = ["age","bmi","children"]
categorical_cols = ["sex", "smoker","region"]

Para fazer OneHotEncoder no Spark, primeiro precisamos transformar valores (`yes/no`) em números (`0/1`). Para isso temos que usar duas funções `StringIndexer` (que irá converter rótulos em valores) e posteriormente a `OneHotEncoder`. O resultado da `StringIndexer` é um vetor esparso. Você sabe dizer a utilidade disso?


Exemplo de um vetor esparso:

```
DenseVector(0, 0, 0, 7, 0, 2, 0, 0, 0, 0)
SparseVector(10, [3, 5], [7, 2])
```

In [63]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

Como sempre temos colunas como `input` e novas colunas como `output`, vamos definir o nome das colunas de output para cada uma das funções

In [66]:
categorical_cols

['sex', 'smoker', 'region']

In [68]:
indexOutputCols = [x+"Index" for x in categorical_cols]
indexOutputCols

['sexIndex', 'smokerIndex', 'regionIndex']

In [70]:
oneHotOutputCols = [x+"OHE" for x in categorical_cols]
oneHotOutputCols

['sexOHE', 'smokerOHE', 'regionOHE']

Agora vamos criar nosso StringIndexer.

**Pergunta**: o que fazemos quando indexamos no treino e no teste há um valor desconhecido?

In [76]:
strIndexer = StringIndexer(inputCols = categorical_cols,
                           outputCols = indexOutputCols,
                           handleInvalid="skip")

E agora criamos nosso `OneHotEncoder`.

**Pergunta**: qual deve ser o input do OneHotEncoder?

In [77]:
oneHotEncoder = OneHotEncoder(inputCols=indexOutputCols,
                              outputCols=oneHotOutputCols)

E também precisamos definir todas as colunas que serão usadas no vector que representará as features.

In [80]:
assemblerInputs = numeric_cols + oneHotOutputCols
assemblerInputs

['age', 'bmi', 'children', 'sexOHE', 'smokerOHE', 'regionOHE']

Criamos o `VectorAssembler`

In [82]:
vecAssembler = VectorAssembler(inputCols=assemblerInputs,
                               outputCol="Features")

Vamos ver se você entendeu: Obtenha o resultado do oneEncoder para o conjunto de treino. Selecione apenas as 20 primeiras linhas, com as colunas `region`, `regionIndex` e `regionOHE` para verificar o resultado.

In [90]:
(oneHotEncoder.fit(strIndexer.fit(train).transform(train))
 .transform(strIndexer.fit(train).transform(train))
 .select("region","regionIndex","regionOHE")
 .show(20))

+---------+-----------+-------------+
|   region|regionIndex|    regionOHE|
+---------+-----------+-------------+
|southeast|        0.0|(3,[0],[1.0])|
|northeast|        1.0|(3,[1],[1.0])|
|northeast|        1.0|(3,[1],[1.0])|
|northeast|        1.0|(3,[1],[1.0])|
|southeast|        0.0|(3,[0],[1.0])|
|northeast|        1.0|(3,[1],[1.0])|
|northeast|        1.0|(3,[1],[1.0])|
|northeast|        1.0|(3,[1],[1.0])|
|southeast|        0.0|(3,[0],[1.0])|
|southeast|        0.0|(3,[0],[1.0])|
|northeast|        1.0|(3,[1],[1.0])|
|southeast|        0.0|(3,[0],[1.0])|
|southeast|        0.0|(3,[0],[1.0])|
|southeast|        0.0|(3,[0],[1.0])|
|southeast|        0.0|(3,[0],[1.0])|
|northeast|        1.0|(3,[1],[1.0])|
|southeast|        0.0|(3,[0],[1.0])|
|northeast|        1.0|(3,[1],[1.0])|
|northeast|        1.0|(3,[1],[1.0])|
|northeast|        1.0|(3,[1],[1.0])|
+---------+-----------+-------------+
only showing top 20 rows



Criamos agora nosso modelo de regressão linear

In [92]:
lr = LinearRegression(labelCol="charges",
                      featuresCol="Features")

Criamos também nosso pipeline, composto pelos estágios:
- stringIndexer
- oheEncoder
- vecAssembler
- lr

In [93]:
pipeline = Pipeline(stages=[strIndexer,oneHotEncoder,
                            vecAssembler,lr])

Aplicamos fit e, posteriormente, transform

In [99]:
pipelineModel = pipeline.fit(train)

In [100]:
pred = pipelineModel.transform(test)

In [102]:
pred.select("age","sex","smoker",
            "Features","charges",
            "prediction").show(5)

+---+------+------+--------------------+-----------+------------------+
|age|   sex|smoker|            Features|    charges|        prediction|
+---+------+------+--------------------+-----------+------------------+
| 18|female|    no|[18.0,24.09,1.0,0...|  2201.0971|   568.92382288308|
| 18|female|   yes|(8,[0,1,2,5],[18....| 18223.4512|25977.071474826116|
| 18|female|    no|(8,[0,1,4,6],[18....|7323.734819| 3049.623568745882|
| 18|female|    no|(8,[0,1,4,6],[18....| 2203.47185| 3383.684533991961|
| 18|female|    no|(8,[0,1,4,5],[18....|  1622.1885|2599.1639479576534|
+---+------+------+--------------------+-----------+------------------+
only showing top 5 rows



Como avaliar agora um modelo que é um pipeline. para esse caso, temos que construir um objeto da classe `RegressionEvaluator`.

In [103]:
from pyspark.ml.evaluation import RegressionEvaluator

In [109]:
regresEval = RegressionEvaluator(
            predictionCol = "prediction",
            labelCol="charges",
            metricName="rmse")

In [110]:
rmse = regresEval.evaluate(pred)
print(f"RMSE: {rmse:.2f}")

RMSE: 5525.14


# Salvando o modelo

É uma boa prática salvar seu modelo para utilizar em outros momentos.

In [112]:
 pipelineModel.write().overwrite().save("./lr-pipelineModel")

# Carregando o modelo

In [113]:
from pyspark.ml import PipelineModel

In [114]:
model = PipelineModel.load("./lr-pipelineModel")

In [115]:
model.stages

[StringIndexerModel: uid=StringIndexer_4499c63a02ec, handleInvalid=skip, numInputCols=3, numOutputCols=3,
 OneHotEncoderModel: uid=OneHotEncoder_ec77508b9b00, dropLast=true, handleInvalid=error, numInputCols=3, numOutputCols=3,
 VectorAssembler_3c296b015721,
 LinearRegressionModel: uid=LinearRegression_45da7b2ccb27, numFeatures=8]

# Otimização de hiperparâmetros e Validação Cruzada

Vamos construir um modelo para nosso problema em questão agora fazendo uso de um `RandomForestRegressor`. Vamos querer otimizar 2 hiperparâmetros (`maxDepth` e `numTrees`), e obter seus valores a partir de um processo de validação cruzada com 5-folds.

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

Para executar o grid, precisamos criar um objeto da classe `ParamGridBuilder`

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder

E criamos também um `RegressionEvaluator`

Para executar a validação cruzada, nós precisamos importar a função `CrossValidator` de `pyspark.ml.tuning`.

In [ ]:
from pyspark.ml.tuning import CrossValidator

E podemos obter nosso modelo a partir do `fit`.

E também podemos verificar todas as trials feitas no processo de otimização.

# Link para referência adicional

Vejam aqui um exemplo de um pipeline de **classificação**.

https://swan-gallery.web.cern.ch/notebooks/SparkTraining/notebooks/ML_Demo1_Classifier.html